### Helpers and Imports

In [34]:
import random
import sys
from typing import Tuple
sys.setrecursionlimit(1000000)

def is_prime(n, k):
    # Miller-Rabin primality test
    # https://gist.github.com/Ayrx/5884790
    
    if n == 2:
        return True
    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True


def generate_big_prime(size):
    k = 40
    p = random.randrange(2 ** (size - 1), 2 ** size - 1)
    if p % 2 == 0:
        p += 1
    while not is_prime(p, k):
        p += 2
    return p


def egcd(a: int, b: int) -> Tuple[int, int, int]:
    # https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Recursive_algorithm_2
    """return (g, x, y) such that a*x + b*y = g = gcd(a, b)"""
    if a == 0:
        return (b, 0, 1)
    else:
        b_div_a, b_mod_a = divmod(b, a)
        g, x, y = egcd(b_mod_a, a)
        return (g, y - b_div_a * x, x)


def modinv(a: int, b: int) -> int:
    # https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm#Modular_inverse
    """return x such that (x * a) % b == 1"""
    g, x, _ = egcd(a, b)
    if g != 1:
        raise Exception('gcd(a, b) != 1')
    return x % b

# Pallier Cryptosystem

https://en.wikipedia.org/wiki/Paillier_cryptosystem

https://doi.org/10.1007%2F3-540-48910-X_16

Security based on the Decisional Composite Residuosity Assumption.

In [67]:
def L(x, n):
    return (x - 1) // n


def gen(security_param):
    # 1. Choose two large prime numbers of equal length
    length = security_param
    p = generate_big_prime(length)
    q = generate_big_prime(length)

    # 2a. Compute n = pq and lambda = lcm(p-1, q-1)    OR
    # 2b. Compute n = pq and lambda = phi(n) = (p - 1) * (q - 1)
    n = p * q
    lambda_ = (p - 1) * (q - 1)
    
    # 3a. Select random integer g from Z_{n^2}^*    OR
    # 3b. Compute g = n + 1
    g = n + 1
    
    # 4a. Ensure n divides the order of g and calculate mu = (L(g^lambda mod n^2))^{-1} mod n    OR
    # 4b. Calculate mu = phi(n)^{-1} mod n = lambda^{-1} mod n
    mu = modinv(lambda_, n)
    
    # Set the public key and secret key
    pk = (n, g)
    sk = (lambda_, mu)
    
    return pk, sk


def enc(pk, m):
    n, g = pk
    
    # 1. Ensure 0 ≤ m < n
    assert (0 <= m),"Message value negative!"
    assert (m < n),"Message value too large. Must be less than %i." %n
    
    # 2. Select random r where 0 < r < n
    r = random.randrange(1,n)
    
    # 3. Compute ciphertext as: c = g^m * r^n mod n^2
    n2 = n**2
    c = pow(g, m, n2) * pow(r, n, n2) % n2
    
    return c


def dec(sk, pk, c):
    n, g = pk
    lambda_, mu = sk
    m = (L(pow(c, lambda_, n**2), n) * mu) % n
    return m

Example of Pallier Cryptosystem in use:

In [68]:
security_param = 30
pk, sk = gen(security_param)
print('pk = (n, g) =', pk)
print('sk = (lambda, mu) =', sk, '\n')

m = 3456667
print('Input message =', m)

c = enc(pk, m)
print('cyphertext =', c)

d = dec(sk, pk, c)
print('Decrypted message =', m)

pk = (n, g) = (876002705271479039, 876002705271479040)
sk = (lambda, mu) = (876002703398440440, 24668984509935488) 

Input message = 3456667
cyphertext = 742401669371856328879855176544184985
Decrypted message = 3456667


# Goldwasser-Micali (GM) Cryptosystem

https://en.wikipedia.org/wiki/Goldwasser%E2%80%93Micali_cryptosystem

https://doi.org/10.1145%2F800070.802212

Security based on the Quadratic Residuoisity Assumption.

In [99]:
def legendre(a, p):
    symbol = pow(a, int((p-1)/2), p)
    if symbol == (p - 1):
        return -1
    else:
        return symbol


def gen(security_param):
    # 1. Choose two large prime numbers
    length = security_param
    p = generate_big_prime(length)
    q = generate_big_prime(length)
    while p == q:
        q = generate_big_prime(length)
    
    # 2. Calculate N = pq
    N = p * q
    
    # 3. Find some non-residue x such that the Legendre symbols satisfy (x/p) = (x/q) = -1
    #    and hence the Jacobi symbol (x/N) = +1
    #    (if N is a Blum integer, x = N - 1 satisfies this)
    x = random.randrange(1, N)
    while legendre(x, p) != -1 or legendre(x, q) != -1:
        x = random.randrange(1, N)
    
    # Set the public key and secret key
    pk = (x, N)
    sk = (p, q)
    return pk, sk

def enc(pk, m):
    pass

def dec(sk, pk, m):
    pass